In [ ]:
!pip install -U finance-datareader;

import pandas as pd
import numpy as np 
import datetime as dt
import pandas_datareader.data as pdr
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import FinanceDataReader as fdr
from datetime import datetime, timedelta 
import math
from scipy.stats import skew, kurtosis
import csv 
import os.path 
import time

Requirement already up-to-date: finance-datareader in /usr/local/lib/python3.6/dist-packages (0.9.14)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# 한국거래소에 있는 주식의 데이터를 가져옴
krx = fdr.StockListing('KOSPI')
krx

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,이현우,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
6,152100,KOSPI,ARIRANG 200,NaN,NaN,NaT,NaN,NaN,NaN,NaN
7,295820,KOSPI,ARIRANG 200동일가중,NaN,NaN,NaT,NaN,NaN,NaN,NaN
8,253150,KOSPI,ARIRANG 200선물레버리지,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6661,215620,KOSPI,흥국 S&P코리아로우볼,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6663,000540,KOSPI,흥국화재,보험업,손해보험,1974-12-05,12월,권중원,http://www.insurance.co.kr,서울특별시
6664,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6665,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [ ]:
!mkdir krx

mkdir: cannot create directory ‘krx’: File exists


In [ ]:
from concurrent import futures
import csv

bad = []
krx_list=krx['Symbol'].tolist() #회사 Symbol만을 추출해서 list로 만든다.


In [ ]:
# Default Date : 2010.01.04 ~ 2020.11.30
default_start = dt.datetime(2010,1,4)
default_end = dt.datetime(2020,11,30)

def download(krx_item):
    sym=krx_item
    df=fdr.DataReader(sym, default_start, default_end)
    dfnew=df.copy()
    dfnew['Symbol']  =sym
    dfnew.to_csv(f"krx/{sym}.csv", quoting=csv.QUOTE_MINIMAL)    
    
workers = min(30,len(krx))
with futures.ThreadPoolExecutor(workers) as executor: 
    res = executor.map(download, krx_list) 

In [ ]:
remove_list=[]

import os.path 

stocks=pd.DataFrame()
for krx_item in krx_list:
  filename=f"krx/{krx_item}.csv"
  status=True
  if os.path.isfile(filename):
    tmp=pd.read_csv(f"krx/{krx_item}.csv",parse_dates=True) 
    if ('Close' in tmp.columns) and ('Date' in tmp.columns):
      tmp.set_index(tmp['Date'],inplace=True,drop=False) 
      tmp_close=tmp['Close'].rename(krx_item) 
      stocks=pd.concat([stocks,tmp_close],axis=1,sort=True) 
      status=False

  if status:
    remove_list.append(krx_item)

for remove_item in remove_list: 
    krx_list.remove(remove_item)

stocks = stocks.loc[:,~stocks.columns.duplicated()];

In [ ]:
# 개별 주식들 close
stocks_close = stocks

# 개별 주식들의 return. log(T시점 종가 / T-1시점 종가)
with np.errstate(divide='ignore'):
    stocks_return = 100 * np.log((stocks_close / stocks_close.shift(1)).astype('float'))

# 코스피 close
kospi = pdr.DataReader('^KS11', 'yahoo', default_start, default_end) #KOSPI
kospi.index = pd.to_datetime(kospi.index).strftime("%Y-%m-%d")  # 기본 format이 time까지 잡혀서 reformat
kospi_close = kospi['Close']
kospi_return = 100 * np.log(kospi_close / kospi_close.shift(1))


In [ ]:
stocks_return

,095570,006840,152100,295820,253150,253160,278420,292750,309170,309210,333940,333950,333960,333970,333980,269530,251590,161510,251600,289670,298340,189400,278620,269540,287180,213630,332610,332620,195970,238670,195980,373530,256450,239660,280920,266550,301400,301410,227830,122090,...,005387,005389,005385,001500,227840,126560,001450,057050,093240,003010,111110,008770,008775,002460,013520,241590,006060,010690,133820,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,081660,005870,079980,005010,069260,215620,000540,000547,000545,003280
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,-1.217330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.251803,-4.530631,-5.873018,-0.854706,NaN,NaN,-0.774197,NaN,3.713211,1.904819,NaN,1.581954,0.100050,3.915091,-2.760527,NaN,-2.432695,-3.376944,NaN,0.379507,-1.481509,0.380721,-1.001573,3.455836,13.895146,NaN,NaN,NaN,NaN,-0.646206,NaN,3.562020,NaN,1.250016,-3.256247,NaN,-3.264385,-1.783992,-1.142870,-0.614846
2010-01-06,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.109069,-0.281294,-0.645580,0.428266,NaN,NaN,4.557751,NaN,0.480275,3.833913,NaN,-0.449439,0.000000,-1.938387,1.138532,NaN,-1.076437,3.249149,NaN,0.378072,0.595240,-1.531658,1.219337,4.863068,6.115442,NaN,NaN,NaN,NaN,1.421507,NaN,3.569447,NaN,3.058342,1.723736,NaN,0.900907,2.176149,0.458716,-0.707342
2010-01-07,NaN,0.916811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.372440,-1.704587,0.645580,-2.378491,NaN,NaN,0.000000,NaN,-0.928320,-2.761517,NaN,-2.971647,-1.207258,0.532388,-3.455238,NaN,-2.631731,-1.157569,NaN,-0.757579,1.472781,-0.774717,5.398544,-3.177986,-5.218575,NaN,NaN,NaN,NaN,0.795760,NaN,-1.175716,NaN,0.300752,-1.723736,NaN,-2.881170,-2.176149,-2.549385,7.847162
2010-01-08,NaN,0.600137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.438041,...,-0.124456,-1.588481,-1.687256,2.804930,NaN,NaN,-0.995033,NaN,1.099290,0.531916,NaN,-0.932408,-3.922071,2.620237,-2.371653,NaN,0.000000,2.806307,NaN,0.757579,2.454997,1.925655,-0.946613,3.501929,3.509132,NaN,NaN,NaN,NaN,0.500726,NaN,-0.659198,NaN,-0.300752,0.690769,NaN,-0.927364,0.000000,0.000000,-0.823728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-24,-0.215285,5.531364,0.555676,-0.267308,1.017211,-1.077763,0.305717,0.000000,1.328293,-1.272552,0.300365,0.145455,0.184196,0.000000,-0.335633,-0.192215,0.315790,0.719946,0.000000,-0.190081,-0.009349,0.465581,-0.014193,0.960227,0.109860,1.793039,-0.159320,-0.796817,0.278940,-0.284226,0.042382,NaN,-0.442674,0.000000,1.317809,0.252207,-0.684153,0.800251,0.416904,0.511719,...,0.345423,0.122324,0.116618,0.000000,0.877199,0.153492,-0.412372,0.808630,0.842465,2.020271,-0.237248,2.793138,0.864737,1.996074,-2.386748,0.000000,-0.396564,1.242252,-2.427757,3.480053,0.156372,0.634923,2.061929,4.143719,-0.224972,4.310324,3.066680,-2.168107,0.714289,0.207039,-2.293679,1.669488,0.246609,-0.273973,1.013180,0.829020,1.25787

In [ ]:
# BUG. 몇몇 날짜는 krx.stock 값은 없고 kospi 값만 있음.

# ks = set(kospi.index.strftime("%Y-%m-%d"))
# # print(ks)
# ss = set(stocks.index)
# # print(ss)
# print(sorted(list(ss - ks)))

In [ ]:
stocks_close

,095570,006840,152100,295820,253150,253160,278420,292750,309170,309210,333940,333950,333960,333970,333980,269530,251590,161510,251600,289670,298340,189400,278620,269540,287180,213630,332610,332620,195970,238670,195980,373530,256450,239660,280920,266550,301400,301410,227830,122090,...,005387,005389,005385,001500,227840,126560,001450,057050,093240,003010,111110,008770,008775,002460,013520,241590,006060,010690,133820,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,081660,005870,079980,005010,069260,215620,000540,000547,000545,003280
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,10331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,43000,37250,41200,23500,NaN,NaN,19450,NaN,2802,3640,NaN,21950,9990,5510,808,NaN,957,3915,NaN,1315,34000,6554,6422,50480,4830,NaN,NaN,NaN,NaN,3726,NaN,7170,NaN,15900,17885,NaN,6850,5090,2200,1142
2010-01-05,NaN,10206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40800,35600,38850,23300,NaN,NaN,19300,NaN,2908,3710,NaN,22300,10000,5730,786,NaN,934,3785,NaN,1320,33500,6579,6358,52255,5550,NaN,NaN,NaN,NaN,3702,NaN,7430,NaN,16100,17312,NaN,6630,5000,2175,1135
2010-01-06,NaN,10206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40350,35500,38600,23400,NaN,NaN,20200,NaN,2922,3855,NaN,22200,10000,5620,795,NaN,924,3910,NaN,1325,33700,6479,6436,54859,5900,NaN,NaN,NaN,NaN,3755,NaN,7700,NaN,16600,17613,NaN,6690,5110,2185,1127
2010-01-07,NaN,10300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12984.0,...,40200,34900,38850,22850,NaN,NaN,20200,NaN,2895,3750,NaN,21550,9880,5650,768,NaN,900,3865,NaN,1315,34200,6429,6793,53143,5600,NaN,NaN,NaN,NaN,3785,NaN,7610,NaN,16650,17312,NaN,6500,5000,2130,1219
2010-01-08,NaN,10362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13041.0,...,40150,34350,38200,23500,NaN,NaN,20000,NaN,2927,3770,NaN,21350,9500,5800,750,NaN,900,3975,NaN,1325,35050,6554,6729,55037,5800,NaN,NaN,NaN,NaN,3804,NaN,7560,NaN,16600,17432,NaN,6440,5000,2130,1209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-24,4640.0,22300,35190.0,9340.0,33595.0,6460.0,8190.0,16185.0,15915.0,14055.0,11670.0,13760.0,13585.0,12875.0,13385.0,10395.0,9515.0,10455.0,10345.0,57815.0,53480.0,15070.0,105680.0,14650.0,18215.0,12380.0,94075.0,101875.0,10770.0,10540.0,11800.0,NaN,11270.0,108295.0,9930.0,9925.0,13110.0,8155.0,26440.0,24490.0,...,87000,81800,85800,12550,11450.0,3260.0,24200,74500.0,2980,5500,8420.0,83500,81300,12650,2070,14850.0,7550,2835,2645.0,3070,32000,15800,7350,81300,22200,64000.0,149000.0,182500.0,140500.0,9670,43100.0,6040,8120.0,7290,24800,9690.0,2800,23200,14850,258
2020-11-25,4600.0,21850,34985.0,9290.0,33165.0,6550.0,8090.0,16110.0,15580.0,14375.0,11630.0,13660.0,13475.0,12860.0,13365.0,10360.0,9455.0,10460.0,10365.0,57665.0,53475.0,15290.0,105675.0,14780.0,18290.0,12670.0,93965.0,101170.0,10870.0,10525.0,11850.0,NaN,11040.0,108295.0,9910.0,9755.0,13040.0,8205.0,26205.0,24425.0,...,85300,81300,84800,12750,11500.0,3225.0,23800,74100.0,2935,5260,8440.0,82700,82000,12500,2040,14550.0,7390,2790,2620.0,3040,31600,15350,7300,79600,21800,62000.0,145000.0,182000.0,134500.0,9390,43650.0,6020,8070.0,7440,24450,9600.0,2875,23550,14050,258
2

코스피

In [ ]:
# Get data as dataframe(['Close', 'Lagged Close', 'Return']) of KOSPI or random stock.
# Default Date : 2010.01.04 ~ 2020.11.30

def getStockData (ticker, start=None, end=None) :
  # Setting default values for start day and end day
  if start is None :
    start = default_start
  if end is None :
    end = default_end

  start = start.strftime("%Y-%m-%d")
  end = end.strftime("%Y-%m-%d")  

  df = pd.DataFrame(columns=['Close', 'Lagged Close', 'Return'])

  if ticker == 'kospi' :
    df['Close'] = kospi['Close']
    df['Lagged Close'] = kospi['Close'].shift(1)
  else :
    try : 
      df['Close'] = stocks_close[ticker]
      df['Lagged Close'] = stocks_close[ticker].shift(1)
    except ValueError :
      return df
  
  df['Return'] = 100 * np.log( df['Close'] / df['Lagged Close'] )

  return df.loc[start:end]

In [ ]:
문제 1. 